<a href="https://colab.research.google.com/github/chinmayayy/Hostile-News-Classification/blob/main/Fake_News_Trial_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd '/content/drive/MyDrive/Colab Notebooks/Project/Dataset' 

/content/drive/MyDrive/Colab Notebooks/Project/Dataset


In [ ]:
ls

'constraint_Hindi_Train - Sheet1.csv'   file1.csv
'Constraint_Hindi_Valid - Sheet1.csv'  'Test Set Complete - test.csv'
 dataset.csv                            Train_Data_01.csv


In [ ]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
data = pd.read_csv('Train_Data_01.csv')

In [ ]:
data.head()

,id,post,labels
0,1,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्क...,"hate,offensive"
1,2,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,non-hostile
2,3,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील...",non-hostile
3,4,@prabhav218 साले जेएनयू छाप कमिने लोग हिन्दुओं...,"defamation,offensive"
4,5,#unlock4guidelines - अनलॉक-4 के लिए गाइडलाइन्स...,non-hostile


In [ ]:
print(f'labels: {data.labels.unique()}')

labels: ['hate,offensive' 'non-hostile' 'defamation,offensive' 'fake' 'hate'
 'offensive' 'fake,hate' 'defamation' 'defamation,hate'
 'defamation,hate,offensive' 'defamation,fake,offensive' 'fake,offensive'
 'defamation,fake' 'defamation,fake,hate' 'fake,hate,offensive'
 'defamation,fake,hate,offensive']


In [ ]:
data["labels"].replace({"hate,offensive": "hate", "defamation,offensive": "offensive", "fake,hate": "fake", "defamation,hate": "hate", "defamation,hate,offensive": "hate", "defamation,fake,offensive": "fake", "fake,offensive": "fake", "defamation,fake": "fake", "defamation,fake,hate" : "fake", "fake,hate,offensive": "fake", "defamation,fake,hate,offensive" : "fake"}, inplace=True)

In [ ]:
print(f'labels: {data.labels.unique()}')

labels: ['hate' 'non-hostile' 'offensive' 'fake' 'defamation']


In [ ]:
data.head()

,id,post,labels
0,1,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्क...,hate
1,2,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,non-hostile
2,3,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील...",non-hostile
3,4,@prabhav218 साले जेएनयू छाप कमिने लोग हिन्दुओं...,offensive
4,5,#unlock4guidelines - अनलॉक-4 के लिए गाइडलाइन्स...,non-hostile


In [ ]:
data['labels'] = data['labels'].apply({'fake':0, 'hate':0, 'offensive':0, 'defamation':0, 'non-hostile':1}.get)

In [ ]:
data.head()

,id,post,labels
0,1,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्क...,0
1,2,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,1
2,3,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील...",1
3,4,@prabhav218 साले जेएनयू छाप कमिने लोग हिन्दुओं...,0
4,5,#unlock4guidelines - अनलॉक-4 के लिए गाइडलाइन्स...,1


In [ ]:
data = data.drop(['id'], axis = 1)

In [ ]:
data = data.dropna()

In [ ]:
x_df = data.iloc[:,:-1].values
y_df = data.iloc[:,-1].values

In [ ]:
x_df

array([['मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्के राम भक्त है और कुछ बाबर के साले है \n\n🙏 जय श्री राम 🙏'],
       ['सरकार हमेशा से किसानों की कमाई को बढ़ाने के लिए नई-नई स्कीमें लाती रहती है, ताकि उन पर ज्यादा आर्थिक बोझ न पड़े.\n\nhttps://t.co/8iy2MJSBAs'],
       ["सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील दीपेश को सुशांत की हत्या के दिन ही क्यों याद आई? देखिए 'पूछता है भारत' अर्नब के साथ रिपब्लिक भारत पर #LIVE :  https://t.co/G945HvzM0Z https://t.co/KfH7xF1IdM"],
       ...,
       ['नौकरी गंवा चुके दोस्त की मदद: नगद के बजाए गिफ्ट कार्ड देने के बारे में सोचें, चुका सकते हैं नेटफ्लिक्स और जिम की फीस; हालचाल जानें, लेकिन ज्यादा नहीं https://t.co/l6kFR8eiBf #HelpYourFriend #Jobless #COVID19 https://t.co/YOKsAyndNa'],
       ['बंगाल में हिन्दू मरे हैं इसलिए मुझे कोई फर्क नहीं पड़ता –#रविश कुमार\n\n'],
       ['रक्षा मंत्री @rajnathsingh ने पूर्व राष्ट्रपति #PranabMukherjee के निधन पर शोक व्यक्त किया\n\nविवरण : https://t.co/aLE7aVtWy9']],
      dtype=object)

In [ ]:
y_df

array([0, 1, 1, ..., 1, 0, 1])

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
count_vectorizer = CountVectorizer()
count_vectorizer.fit_transform(x_df.ravel())
freq_term_matrix = count_vectorizer.transform(x_df.ravel())
tfidf = TfidfTransformer(norm = "l2")
tfidf.fit(freq_term_matrix)
tf_idf_matrix = tfidf.fit_transform(freq_term_matrix)
print(tf_idf_matrix)

  (0, 8560)	0.4788011212620214
  (0, 8544)	0.3879931928102296
  (0, 8516)	0.3717766973795364
  (0, 8340)	0.4825209264567969
  (0, 7871)	0.44400494428498294
  (0, 7479)	0.22809213243442533
  (1, 9308)	0.19600563451817907
  (1, 9226)	0.19560712213918902
  (1, 8902)	0.308813199885272
  (1, 8482)	0.23789465044650407
  (1, 8381)	0.11667703811056195
  (1, 8361)	0.24848894219184678
  (1, 8256)	0.5536481131028202
  (1, 7551)	0.20530014891042014
  (1, 7236)	0.2204035893539649
  (1, 7067)	0.2245539049226613
  (1, 3064)	0.0938824319126176
  (1, 1950)	0.09412100649669321
  (1, 912)	0.4802610172295583
  (2, 9289)	0.21430106846831662
  (2, 8837)	0.292458433979706
  (2, 8381)	0.11213205627383631
  (2, 8377)	0.28466239178990255
  (2, 8291)	0.31560931796731323
  (2, 7834)	0.18953687837728742
  :	:
  (5725, 7479)	0.09422577934169422
  (5725, 6652)	0.354501035525315
  (5725, 3776)	0.354501035525315
  (5725, 3468)	0.354501035525315
  (5725, 3064)	0.13859721333472189
  (5725, 2982)	0.354501035525315
  (572

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(tf_idf_matrix,y_df, test_size=0.25, random_state=0)

##Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
Accuracy = logreg.score(x_test, y_test)

In [ ]:
print(Accuracy*100)

94.27374301675978


##Naive-Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
NB = MultinomialNB()
NB.fit(x_train, y_train)
Accuracy = NB.score(x_test, y_test)

In [ ]:
print(Accuracy*100)

89.10614525139665


##Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(x_train, y_train)
Accuracy = clf.score(x_test, y_test)

In [ ]:
print(Accuracy*100)

91.75977653631286


In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(criterion='entropy')
dtc.fit(x_train, y_train)
y_pred = dtc.predict(x_test)

Creating a confusion matrix to determine the accuracy


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[611,  60],
       [ 63, 698]])

In [ ]:
plt.clf()
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Wistia)
classNames = ['Negative','Positive']
plt.title('Hostile v/s Non-hostile Data')
plt.ylabel('True label')
plt.xlabel('Predicted label')
var = np.arange(len(classNames))
plt.xticks(var, classNames, rotation=45)
plt.yticks(var, classNames)
s = [['TN','FP'], ['FN', 'TP']]
for i in range(2):
    for j in range(2):
        plt.text(j,i, str(s[i][j])+" = "+str(cm[i][j]))
plt.show()

##Passive-Aggressive Classifier


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import PassiveAggressiveClassifier
pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(x_train,y_train)

PassiveAggressiveClassifier(C=1.0, average=False, class_weight=None,
                            early_stopping=False, fit_intercept=True,
                            loss='hinge', max_iter=50, n_iter_no_change=5,
                            n_jobs=None, random_state=None, shuffle=True,
                            tol=0.001, validation_fraction=0.1, verbose=0,
                            warm_start=False)

In [ ]:
y_pred=pac.predict(x_test)
score=accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 96.55%


Conclusion: Best accuracy achieved using linear regression